In [11]:
import pandas as pd

In [12]:
data_dir = '../data/random_data'
tracks = pd.read_csv('{}/tracks.csv'.format(data_dir), index_col=0)
artists = pd.read_csv('{}/artists.csv'.format(data_dir), index_col=0)
albums = pd.read_csv('{}/albums.csv'.format(data_dir), index_col=0)
transactions = pd.read_csv('{}/transactions.csv'.format(data_dir), index_col=0)
playlists = pd.read_csv('{}/playlists.csv'.format(data_dir), index_col=0)

In [13]:
def get_full_dataset(transactions, tracks, playlists):
    full_dataset = pd.merge(transactions, tracks, how='left', on='trackid')
    full_dataset = pd.merge(full_dataset, playlists, how='left', on='pid')
    return full_dataset

In [14]:
full_dataset = get_full_dataset(transactions, tracks, playlists)
full_dataset['rating'] = 1
full_dataset.head()

,pid,trackid,popular,artistid,albumid,name,num_followers,rating
0,822032,2283,3384,1151,1618,party time,1,1
1,822032,2288,4339,1152,1622,party time,1,1
2,822032,907,4992,500,687,party time,1,1
3,822032,2289,1822,503,690,party time,1,1
4,822032,910,2160,503,690,party time,1,1


$R$ - матрица плейлист-трек
$X$ - матрица плейлист-признак
$C$ - матрица популярность-трек
$U, V$ - матрицы профилей 
$$R = XUVC$$
$$(X^TX)^{-1}X^TRC^T = UV$$
$$\hat{r}_{ui} = \mathbf{x}_u^TUV\mathbf{c}_i$$

In [ ]:
    def get_training_content_matrix(self):
        self.get_content_shape()
        idx_userid = self.data.training[self.data.fields[0]].values
        val = np.ones(self.data.training.shape[0])
        
        i = 0
        features = []
        
        for col in self.train_content.columns:
            idx_feature = self.train_content[col].values
            shp = (idx_userid.max() + 1, 
                   self.content_shape[col])
        
            features_new = sparse.csr_matrix((val, (idx_userid, idx_feature)), 
                                             shape=shp)
            
            if i == 0:
                features = features_new
            else:
                features = sparse.hstack((features, features_new))
            
            i += 1
        
        return features

In [15]:
def get_playlist_artist_album_matrix(data, content_columns, user_col):
    idx_userid = data[user_col].values
    val = np.ones(data[user_col].shape[0])
    
    i = 0
    features = []
    
    for col in content_columns:
        idx_feature = data[col].values
        shp = (idx_userid.max() + 1,
               idx_feature.max() + 1)
        
        features_new = sparse.csr_matrix((val, (idx_userid, idx_feature)), 
                                         shape=shp)
        
        if i == 0:
            features = features_new
        else:
            features = sparse.hstack((features, features_new))
            
        i+=1
    
    return features

def get_playlist_name_matrix(playlists):
    playlists['name'] = [str(playlists['name'].values[i]).lower() 
                         for i in range(playlists.shape[0])]
    names = playlists['name'].values
    for i in range(names.shape[0]):
        names[i] = re.compile('[^a-zA-Z0-9 ]').sub('', names[i])
    playlists['name'] = names
    words_in_names = []
    for i in range(names.shape[0]):
        words_in_names += names[i].split(' ')
    stop_words = stopwords.words('english') + ['music', 'song', 'songs', 'playlist', '', 'good']
    words_in_names = np.array([word for word in words_in_names if word not in stop_words])
    bag_of_words = CountVectorizer()
    bag_of_words.vocabulary_ = words_in_names
    print(words_in_names)
    X = bag_of_words.fit_transform(playlists.name.values)
    return X

def get_popularity_track_matrix(data):
    enc = OneHotEncoder()
    enc.fit(data[['', 'name']])

SyntaxError: invalid syntax (<ipython-input-15-f3f8e1980480>, line 1)

(7657, 8)

In [17]:
playlists.shape

(7657, 3)